First we import the library, forcing reload from local version (not pip install). 

In [1]:
import os
import sys
import importlib
import matplotlib.pyplot as plt


# Get the absolute path to the directory containing the local module
stlrom_path = os.path.abspath('../build')

# Insert the local module path at the beginning of sys.path
sys.path.insert(0, stlrom_path)

import stlrom

# Reload the module to ensure it is loaded from the new path
importlib.reload(stlrom)

import stlrom as sr
import numpy as np


# Signals


## Plotting function


In [2]:
import matplotlib.pyplot as plt

def plot_samples(signal,label=None, fig=None):
    samples_list = signal.get_samples_list()

    if fig is None:
        fig = plt.figure(figsize=(15, 5)).gca()

    if label is None:
        label = f'Signal {len(fig.get_lines()) + 1}'

    fig.set_xlabel('Time')
    fig.set_ylabel('Value')
    fig.set_title('Signal Plot')
    fig.grid(True)
        

    def plot_sample(s, dt, color=None):
        if color is None:
            l, = fig.plot(s.time, s.value, marker='o')
            color = l.get_color()
            l = fig.plot([s.time, s.time+dt], [s.value, s.value_at(s.time+dt)], color=color, linestyle='-')
        else:
            l = fig.plot(s.time, s.value, marker='o', color=color)
            l = fig.plot([s.time, s.time+dt], [s.value, s.value_at(s.time+dt)], color=color, linestyle='-')
        return color,l
    
    
    s = samples_list[0]
    sn = samples_list[1]
    dt = sn.time-s.time
        
    col,l = plot_sample(s, dt)
    l[0].set_label(label)
    
    for i in range(1, len(samples_list)-1):
        s = samples_list[i]
        sn = samples_list[i+1]
        dt = sn.time-s.time
        plot_sample(s,dt,color=col)
    
    plot_sample(sn, signal.end_time-sn.time, col)
    #fig.plot(sn.time, sn.value, marker='o', color=col)
    fig.legend()
    fig.figure.canvas.draw()

    return fig

def print_signal(signal):
    for s in signal.get_samples_list():
        s.print_sample()
    print('begin_time ', signal.begin_time)
    print('end_time    ',signal.end_time)

# Drivers

In [31]:
stl_parser = stlrom.STLDriver()

stl_parser.parse_string(""" signal p, q 
p_    := p[t]>.5
not_p := p[t]<.5                                                                             
q_ := q[t]>.5
not_q := q[t]<.5
f1 := not_q until_[0, 10] (not_p and not_q)
f2 := not( p_ until_[0, 10] q_ )
""")

stl_parser.add_sample([0, 1, 0])
stl_parser.add_sample([7, 0, 0])
stl_parser.add_sample([19, 1, 1])
stl_parser.add_sample([37, 0, 1])
stl_parser.add_sample([69, 1, 1])
stl_parser.add_sample([100, 1, 1]) 



In [32]:
stl_parser.set_semantics("LEFT_TIME")
stl_parser.set_interpol("PREVIOUS")

interpol:PREVIOUS
prev


In [33]:
f1_left = stl_parser.get_monitor("f1")
f2_left = stl_parser.get_monitor("f2")


In [34]:
stl_parser.set_semantics("RIGHT_TIME")
f1_right = stl_parser.get_monitor("f1")
f2_right = stl_parser.get_monitor("f2")


In [35]:
print("f1 left robustness:", f1_left.eval_rob())
print("f2 left robustness:", f2_left.eval_rob())
print("f1 right robustness:", f1_right.eval_rob())
print("f2 right robustness:", f2_right.eval_rob())

f1 left robustness: 1e-14
f2 left robustness: 1e-14
f1 right robustness: 3.0
f2 right robustness: 3.0
